In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'customer-segmentation-tutorial-in-python:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F42674%2F74935%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240526%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240526T202633Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3b03ca84987ad473fe60c90cf037b54603ee6afc26487439c654b0d160a87b059b856f616098bfb56812641fb02a0cb5acc34499f291c43cd8d5c6b65c41f61fc245278eb57bcaab0bff240d22d18b1c53015693e0df5f902cc9c8e88c6134135d10513a5166e6cfcd06666903aa8e18c666b2df1b0f85b5eca6a2ae57a9330cbfa0e3024adc6588102178716bdfda286578facc755b781b61d889a257c8f112c53969df1bf9512b4f305e95ae8f29a31c74c21e492564f941676403debbf10d79a8e320511797cbf845785bb4c5c6c6cf9a8920cd6b4de07dbb6121189103f05b720076cbd07347079f47a1af158a2d2a56c416913f38040d3515e998df8212'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# **Import Libraries :** #

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier ,Pool
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import joblib

# **Reading Data :** #

In [ ]:
data = pd.read_csv("/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv")

# **EDA :**

In [ ]:
data.head(7)

In [ ]:
data.tail(7)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
cols=data.columns
for col in cols:
    print(data[col].value_counts(),"\n")

#  **EDA With Visualization :** #

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
sns.pairplot(data)

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
for col in data.select_dtypes(include=['int', 'float']).columns:
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.set_title(f'Boxplot of {col}')
    bp = sns.boxplot(data=data, x=col, ax=ax)
    plt.show()
    plt.close(fig)

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
sns.displot(data=data, x="Gender")

In [ ]:
sns.displot(data, x = 'Age',hue='Gender', kind='kde')

In [ ]:
sns.set_palette("Set2")
sns.distplot(data['Age'], kde=True, bins=30)
plt.title('Distribution Plot of the Gender of customers')
plt.show()

In [ ]:
sns.set_palette("Set2")
sns.distplot(data['Annual Income (k$)'], kde=True, bins=20)
plt.title('Distribution Plot of the Anual income of customers')
plt.show()

In [ ]:
sns.set_palette("Set2")
sns.distplot(data["Spending Score (1-100)"], kde=True, bins=30)
plt.title('Distribution Plot of the Spending Score of customers')
plt.show()

In [ ]:
custom_palette = "Set2"
sns.violinplot(x='Age', y='Gender', data=data, palette=custom_palette)
plt.title('Violinplot of gender with the age ')
plt.show()

In [ ]:
custom_palette = "Set2"
sns.violinplot(x="Spending Score (1-100)", y='Gender', data=data, palette=custom_palette)
plt.title('Violinplot of gender and the spending scores')
plt.show()

In [ ]:
custom_palette = "Set2"
sns.violinplot(x="Annual Income (k$)", y='Gender', data=data, palette=custom_palette)
plt.title('Violinplot of gender and the annual income')
plt.show()

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
plt.figure(figsize=(8, 6))
data['Gender'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=sns.color_palette()
                                   , startangle=140)
plt.title("Distribution of Gender")
plt.ylabel("")
plt.axis('equal')
plt.show()

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
plt.figure(figsize=(20,10))
sns.boxplot(data=data, x="Age", y="Annual Income (k$)")
plt.show()

In [ ]:
custom_palette = "Set2"
sns.set_palette(custom_palette)
plt.figure(figsize=(20,10))
sns.boxplot(data=data, x="Age", y="Spending Score (1-100)")
plt.show()

# **Pre-Processing Operations :**

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
# Dropping the customer id columns because it will be not useful and maybe has effects on the models
data.drop(columns=["CustomerID"],inplace=True)

In [ ]:
Features=data.drop(columns=["Gender"])
Goal=data["Gender"]

In [ ]:
Features.head(10)

In [ ]:
Goal.head(10)

In [ ]:
Scaled_Features = MaxAbsScaler().fit_transform(Features)
Scaled_Features = pd.DataFrame(Scaled_Features, columns=Features.columns)

In [ ]:
Scaled_Features.head()

In [ ]:
# Saving Scaled Features as CSV File
Scaled_Features.to_csv('Scaled Features.csv', index=False)

In [ ]:
Ftrain,Ftest,Gtrain,Gtest=train_test_split(Scaled_Features,Goal,test_size=0.1,random_state=0)

In [ ]:
Ftrain.head()

In [ ]:
Ftest.head()

In [ ]:
Gtrain.head()

In [ ]:
Gtest.head()

# **Applying Grid Search Model on Decision Tree Classifier :**

In [ ]:
DTC = DecisionTreeClassifier()

DTC_Params = {
    'max_depth' : [3,5,7,9,10,11,12],
    'min_samples_leaf' :list(range(2,20)),
    'criterion': ['gini', 'entropy'],
    'min_samples_split' : list(range(2,20))
}

GRS = GridSearchCV(DTC, DTC_Params, cv = 7)
GRS.fit(Ftrain, Gtrain)

print(GRS.best_params_)
print(GRS.best_score_)

In [ ]:
DTC=DecisionTreeClassifier(criterion= 'entropy', max_depth= 12, min_samples_leaf= 2, min_samples_split= 5)
DTC.fit(Ftrain,Gtrain)
DTC_Pred=DTC.predict(Ftest)
DTC_Pred_Train=DTC.predict(Ftrain)

In [ ]:
test_accuracy = accuracy_score(Gtest, DTC_Pred)
print(f"Accuracy Score for the test group: {test_accuracy*100}\n")
train_accuracy = accuracy_score(Gtrain, DTC_Pred_Train)
print(f"Accuracy Score for the train group: {train_accuracy*100:.2f}\n")

In [ ]:
print("Classification Report for the test group : \n")
print(classification_report(Gtest,DTC_Pred),"\n\n")
print("Classification Report for the train group (to check if there is any overfitting): \n")
print(classification_report(Gtrain,DTC_Pred_Train),"\n\n")

In [ ]:
CM_DTC=confusion_matrix(Gtest,DTC_Pred)
print(CM_DTC)
sns.heatmap(CM_DTC,annot=True,cmap="Set3")
plt.show()

# **Applying Grid Search Model on CatBoosting Classifier :**

In [ ]:
CB_params = {
    'learning_rate': [0.05, 0.1],
    'depth': [2,3,4,5,6,7],
    'iterations': [100,150,250,300,350],
    'bagging_temperature': [0.5, 1.0]
}

CB = CatBoostClassifier()
GRS = GridSearchCV(CB, CB_params, cv=7)
GRS.fit(Ftrain, Gtrain)

# Print the best parameters and best score
print(GRS.best_params_)
print(GRS.best_score_)

In [ ]:
CB=CatBoostClassifier(bagging_temperature=0.5,depth=4 ,iterations=150,learning_rate=0.1)
CB.fit(Ftrain,Gtrain)
CB_Pred=CB.predict(Ftest)
CB_Pred_Train=CB.predict(Ftrain)

In [ ]:
test_accuracy = accuracy_score(Gtest, CB_Pred)
print(f"Accuracy Score for the test group: {test_accuracy*100:.4f}\n")
train_accuracy = accuracy_score(Gtrain, CB_Pred_Train)
print(f"Accuracy Score for the train group: {train_accuracy*100:.4f}\n")

In [ ]:
print("Classification Report for the test group : \n")
print(classification_report(Gtest,CB_Pred),"\n\n")
print("Classification Report for the train group (to check if there is any overfitting): \n")
print(classification_report(Gtrain,CB_Pred_Train),"\n\n")

In [ ]:
CM_CB=confusion_matrix(Gtest,CB_Pred)
print(CM_CB)
sns.heatmap(CM_CB,annot=True,cmap="Set3")
plt.show()

# **Models Evaluation and Choosing The Final Used Model :**

#### After evaluating both models, the CatBoosting model demonstrates superior performance over the Decision Tree model. The CatBoosting model achieved an accuracy of 55% on the test set compared to the Decision Tree's 40%. Additionally, CatBoosting shows better precision and recall for both classes, indicating a more balanced and robust model. While both models exhibit some degree of overfitting, CatBoosting's metrics are more favorable, with a higher f1-score and accuracy on the test set. Therefore, CatBoosting is the recommended model for deployment.

# **Saving CatBoosting Model :**

In [ ]:
joblib.dump(CB,"CatBoostingModel (Mall Customer Segmentation Data) FinalModel - GDSC #4 Task.sav")